In [81]:
import pandas as pd
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.trainer as trainer
import torch.utils.trainer.plugins
from torch.autograd import Variable
import numpy as np
import os

from imagedataset import ImageDataset

In [2]:
%matplotlib inline
def show(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')

In [88]:
# train_dir = "train"
train_dir = "sample"
use_cuda = False
batch_size = 2
print('Using CUDA:', use_cuda)

Using CUDA: False


In [4]:
data_path = "data/galaxies/"

def dataframe_from_csv():
    df=pd.read_csv(data_path + "classes.csv", sep=',')
    df.set_index("GalaxyID", inplace=True)
    return df

In [84]:
import importlib
import imagedataset
importlib.reload(imagedataset)

# Data loading code
traindir = os.path.join(data_path, train_dir )
valdir = os.path.join(data_path, 'valid') 
testdir = os.path.join(data_path, 'test')

targets = dataframe_from_csv();

# pytorch way of implementing fastai's get_batches, (utils.py)
def get_data_loader(dirname, shuffle=True, batch_size = 64):
    image_dataset = ImageDataset(dirname, targets)
    return torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, 
                                       shuffle=shuffle, pin_memory=use_cuda), image_dataset

train_loader, train_dataset = get_data_loader(traindir, batch_size=batch_size)

print('Images in train folder:', len(train_dataset))

Images in train folder: 10


In [27]:
# Load the model
model = models.resnet50(pretrained=True)

In [53]:
# Finetune by replacing the last fully connected layer and freezing all network parameters
for param in model.parameters():
    param.requires_grad = False

# Replace the last fully-connected layer matching the new class count
num_classes = len(targets.columns)
print('Using {:d} classes: {}'.format(num_classes, targets.columns))
expansion = 4 # TODO use Bottleneck.expansion instead
model.fc = nn.Sequential(nn.Linear(512 * expansion, num_classes), nn.Softmax())

Using 37 classes: Index(['Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1',
       'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3',
       'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3',
       'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6',
       'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1',
       'Class10.2', 'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3',
       'Class11.4', 'Class11.5', 'Class11.6'],
      dtype='object')


In [54]:
# define loss function (criterion) and optimizer
criterion = nn.MSELoss()
# enable cuda if available
if(use_cuda):
    model.cuda()
    criterion.cuda()
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

In [55]:
def getTrainer():
    # fine-tune with new classes
    t = trainer.Trainer(model, criterion, optimizer, train_loader)
    t.register_plugin(trainer.plugins.ProgressMonitor())
    t.register_plugin(trainer.plugins.LossMonitor())
    t.register_plugin(trainer.plugins.TimeMonitor())
    t.register_plugin(trainer.plugins.Logger(['progress', 'loss', 'time']))
    
    # Requires a monkey patched version of trainery.py that calls
    # input_var = Variable(batch_input.cuda())    # Line 57
    # target_var = Variable(batch_target.cuda())  # Line 58
    return t

In [93]:
t = getTrainer()
epochs = 1
model.train()
t.run(epochs)

progress: 1/5 (20.00%)	loss: 0.0577  (0.0173)	time: 0ms  (0ms)
progress: 2/5 (40.00%)	loss: 0.0905  (0.0393)	time: 606ms  (182ms)
progress: 3/5 (60.00%)	loss: 0.0666  (0.0475)	time: 598ms  (307ms)
progress: 4/5 (80.00%)	loss: 0.0755  (0.0559)	time: 586ms  (390ms)
progress: 5/5 (100.00%)	loss: 0.0614  (0.0575)	time: 584ms  (449ms)
################################################################################
Epoch summary:
loss: 0.0703	time: 475ms
################################################################################


In [ ]:
# Load validation data
val_loader, val_dataset = get_data_loader(valdir, shuffle=False, batch_size=batch_size)

In [64]:
import sys

def get_error(val_loader):
    # Process each mini-batch and accumulate all correct classifications
    num_batches = sum(1 for b in enumerate(val_loader))
    batches = enumerate(val_loader)
    error2 = 0
    for i, (images, labels) in batches:
        sys.stdout.write('\rBatch: {:d}/{:d}'.format(i + 1, num_batches))
        sys.stdout.flush()
        if use_cuda:
            images = images.cuda()
        predictions = model(Variable(images))
        error2 += labels.sub(predictions.data).pow(2).sum()
    # Avoid carriage return
    print('')
    return np.sqrt(error / len(val_loader.dataset.images_targets))

In [ ]:
model.eval()
print('RMSE for validation set: {}'.format(get_error(val_loader)))

In [ ]:
# Load test data
test_loader, test_dataset = get_data_loader(testdir, shuffle=False, batch_size=batch_size)

In [94]:
import sys

def predict(loader):
    # Process each mini-batch and accumulate all correct classifications
    all_predictions_df = pd.DataFrame(data=None, columns=targets.columns,index=targets.index)
    # Drop all rows
    all_predictions_df.drop(targets.index, inplace=True)
    num_batches = sum(1 for b in enumerate(loader))
    batches = enumerate(loader)
    current_image = 0
    for i, (images, labels) in batches:
        sys.stdout.write('\rBatch: {:d}/{:d}'.format(i + 1, num_batches))
        sys.stdout.flush()
        if use_cuda:
            images = images.cuda()
        predictions = model(Variable(images))
        batch_predictions = predictions.data.cpu().numpy()
        for row_prediction in batch_predictions:
            image_id = loader.dataset.images_idx_to_id[current_image]
            all_predictions_df.loc[image_id] = row_prediction
            current_image += 1
    # Avoid carriage return
    print('')
    return all_predictions_df

def save_kaggle_predictions(loader):
    all_predictions_df = predict(loader)
    all_predictions_df.to_csv(data_path + "predicted-resnet.csv")
    return all_predictions_df
    

In [97]:
from IPython.display import FileLink
model.eval()
save_kaggle_predictions(train_loader)
FileLink(data_path + "predicted-resnet.csv")

Batch: 5/5


/Users/rodrigo/Libs/fastai-courses/deeplearning1/nbs/data/galaxies/predicted-resnet.csv